<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-UT/blob/main/DataAlgo_UT(009)_ShortestPathNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.3. Ｎ対Ｎの最短経路問題

ここではグラフ中の任意の２頂点の間の最短経路問題とそれを解くFloydのアルゴリズムについて学ぶ．

**いつもの約束**  
１つのコードセルだけの実行は Ctrl + Enter．  
エディタで「インデント幅（スペース）は4で表示」「行番号を表示」「インデントガイドを表示」．  
挿入図は Google Colaboratory  以外では見れない可能性あり．  
内部では日本語はUTF-8で表現されている．


# 準備

インスタンスに接続し起動する．  
下記の手順でGoogle Driveをマウントする．  
マウント先に移動し，作業フォルダとする．  
これによって，インスタンスがリセットされてもGoogle Drive内にファイルが保存されるようにする．

In [ ]:
!echo "Google Driveをマウントします"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "今回の作業用フォルダを作成しそこに移動します"
%cd /content/drive/My\ Drive/
%mkdir -p 202104_DataAlgo/DA2021_009
%cd       202104_DataAlgo/DA2021_009
!ls
!echo "日本時間表示"
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime
!date

本節での説明には，4.2.節と同じくグラフ6を利用する．  
頂点数は8．無向グラフ．ループなし．重みは全て正．  
自己ループ（ある頂点から同じ頂点への辺）の重みは 0 とする．  
プログラムの解析用に，グラフ7, グラフ8も用意しておく．これらも4.2.節と同じである．



![008-GraphG6](https://user-images.githubusercontent.com/45651568/119325594-fc232500-bcbb-11eb-80b4-f25270057bad.png)


In [ ]:
%%writefile graph6.h
// 8 nodes, undirected, no loop, positive weight.
// NC means no edges.
// NC will be treated as "inifinity" on searching the shortest path. 
#define N 8
#define NC 9999 // this big value means both no path and infinity 
int edge[N][N] = {
//     0   1   2   3   4   5   6   7
	{  0,100, NC, NC,145,200, NC, NC}, // 0
	{100,  0, 23, NC, NC, 80, NC, 70}, // 1
	{ NC, 23,  0, 21, NC, 45, 37, NC}, // 2
	{ NC, NC, 21,  0, NC, NC, 18,  5}, // 3
	{145, NC, NC, NC,  0, 20, NC, NC}, // 4
	{200, 80, 45, NC, 20,  0,  2, NC}, // 5
	{ NC, NC, 37, 18, NC,  2,  0,  1}, // 6
	{ NC, 70, NC,  5, NC, NC,  1,  0}  // 7
};

In [ ]:
%%writefile graph7.h
// 4 nodes, directed, positive weight.
// NC means no edges.
// NC will be treated as "inifinity" on searching the shortest path. 
#define N 4
#define NC 9999 // this big value means no path
int edge[N][N] = {
//     0   1   2   3
	{  0, 10, NC, 40}, // 0
	{ NC,  0, NC, 20}, // 1
	{ NC, NC,  0, NC}, // 2
	{ NC, NC, NC,  0}, // 3
};

In [ ]:
%%writefile graph8.h
// 12 nodes, directed, positive weight.
// NC means no edges.
// NC will be treated as "inifinity" on searching the shortest path. 
#define N 12
#define NC 9999 // this big value means no path
int edge[N][N] = {
    { 0, 4,NC,NC, 3,NC,NC,NC,NC,NC,NC,NC},
    { 4, 0, 1,NC,NC,NC, 2,NC,NC,NC,NC,NC},
    {NC, 1, 0, 1,NC,NC,NC, 2,NC,NC,NC,NC},
    {NC,NC, 1, 0,NC,NC,NC,NC,NC,NC,NC,NC},
    { 3,NC,NC,NC, 0, 1,NC,NC, 1,NC,NC,NC},
    {NC,NC,NC,NC, 1, 0, 2,NC,NC, 2, 3,NC},
    {NC, 2,NC,NC,NC, 2, 0,NC,NC,NC, 3,NC},
    {NC,NC, 2,NC,NC,NC,NC, 0,NC,NC,NC,NC},
    {NC,NC,NC,NC, 1,NC,NC,NC, 0,NC,NC,NC},
    {NC,NC,NC,NC,NC, 2,NC,NC,NC, 0,NC,NC},
    {NC,NC,NC,NC,NC, 3, 3,NC,NC,NC, 0, 1},
    {NC,NC,NC,NC,NC,NC,NC,NC,NC,NC, 1, 0}
};

# Ｎ対Ｎの最短経路問題を解くfloydのアルゴリズム

**内容**

最短経路問題の内容については，4.2.節を参照すること．  

**別解としてDijkstraのアルゴリズムの再利用**

新しいアルゴリズムを用意する前に，単純な方法を考えてみよう．
shortest_dijkstra プログラムを，開始頂点を1からNまで変えながら全て実行すれば解は得られるはずである．
この考え方は実に率直で，しかもオーダー表現による議論では，後で説明する floydのアルゴリズムとも変わらない．

ただ，floydのアルゴリズムは，これまたプログラミングスタイルとして（Diskstraのアルゴリズムと対極的に）劇的に洗練されている例なので，本授業で取り上げて説明する．（換言すれば人間からすれば最も理解しにくいアルゴリズムかもしれない．）

**Floydのアルゴリズム**

Ｎ対Ｎの最短経路問題を解くアルゴリズムとして，Floyd（フロイド）のアルゴリズムを学習する．

Floydのアルゴリズムは，動的計画法(Dynamic programming)の１つとして知られる．

>**（動的計画法）**
>
>動的計画法の詳細は本授業では省くが，簡単に言えば，下記のようになる．
>
>1. 元の問題を分割する
>2. 分割した問題のそれぞれで解を求める
>3. それぞれの問題と解をペアにして覚えておく．
>4. 以降同じ問題が来たときは覚えておいた解を引用する（改めて解かない）
>5. 上記1-4を可能な限り繰り返す．つまり，元問題を分割して用意した部分問題を解く際に，さらに分割した部分部分問題を解く，とする．
>
>残念ながら，日常生活で動的計画法を使った処理をヒトが思いつくことはほぼない．これは，動的計画法の立案には緻密な問題検討と解法の設計が必要であり，かつ多数の問題と解のペアを覚えておく必要があるからである．（人は多くのことを正確に覚えるのは苦手である）
>
>その代わり，がんばって動的計画法を立案できると，劇的なほどアルゴリズムが洗練され見通しがよくなり，多くの場合は実行時に様々な利点が得られることが多い．
>
>設計上の難点は，2.の分割を均等にしつつ，5.で再帰的に行えるようにできるか，である．

Floydのアルゴリズムにおいて，核となる部分は実は単純である．  
今，探索の途中にあるとし，頂点Aから頂点Bに至る（今までに見つかっている暫定最短）経路値を W(A,B) とする．
頂点A,B,Cについて， 

> W(A,C) + W(C,B) < W(A,B) 

が成立する，つまり頂点Cを経由したほうが経路値が小さくなるのなら， W(A,B) の値を W(A,C) + W(C,B) の値で更新する．  
これを繰り返すだけである．

問題は，これをいつどの順で行うのか，である．  
上手な順番を思いつかないと，いつまで経っても答えが得られなかったり，下手をすると堂々巡りするかもしれない．

Floydのアルゴリズムでは，これを，「段階的に経由してよい頂点を増やしていく」ことで実現する．  
この１段１段が部分問題に相当する．  
さらに，部分問題の解を覚えるのに配列を用いるが，これを巧みに使うことで，解を覚えておくのに必要な領域を極限まで小さくかつ簡潔な表現にしている．

それでは実際のアルゴリズムを見てみよう．

1. 全ての頂点対A,Bについて，AからBへの暫定経路値 W(A,B) を無限大とする．
2. 全ての頂点対A,Bについて，隣接している（直行できる）場合のみ， W(A,B) を edge(A,B) の値で更新する．
3. 新たに経由してもいい頂点kを１つ選ぶ．(実装では k = 1,2,3, ..., N の順に選択）
4. 全ての頂点対A,Bについて，W(A,K) + W(K,B) < W(A,B) のときのみ W(A,B) を W(A,K) + W(K,B) の値で更新する．
5. 上記3.と4.を新たに経由してもいい頂点が無くなるまで繰り返す．

グラフ6による実行の様子を次図に示す．

ステップ1.が終わったところが「直通のみを認めた結果」である．
各要素はSrc頂点からDst頂点へ至る経路の暫定最小経路値 W(Src,Dst) が入っている．

次に，頂点0 を新たに経由頂点として認めることを考える．この時，

* W(Src,Dst)
* W(Src,0) + W(0, Dst)

という二つの暫定経路値を比較し，更新した場合は桃色で表示している．
このとき注意するべきなのは，頂点0から始発する経路と，頂点0に終着する経路は，その暫定最小経路値が変わるはずがないという事実である．つまり，図において，太枠で囲まれた部分は，値の更新が絶対に発生しない．

> 仙台と名古屋の間の出張を安く行くように考えている時，それまで直通出張しか許されなかったのが「東京も経由していいよ」と言われれば検討するが，「仙台を経由してもいいよ」と言われても仙台はそもそも出発地なので「何意味のないことを言ってるの？」となろう（仙台発着でない出張を考えている者には意味のある言葉なのだが）．

暫定経路値の２つめの和のほうは，その変わるはずのない経路**のみ**を組み合わせて算出している点にも注意する．次図中では， W(1,4) はそれまで NC であったが， 100+145=245 という新しい値で更新している．このような暫定経路値の吟味と更新を，N*Nのどのセルをどの順で更新しても，この結果が他のセルの実行結果で影響を受けることはない．（その意味でN*Nのセルの更新はN*N並列で実行可能）

あとは，これを繰り返していく．次の「頂点1を経由頂点として追加した結果」では，その前提となる暫定経路値は W(Src,1) と　W(1,Dst) である．このどちらも「直行ないし頂点0の経由迄が認められたうちでの最小経路値」が示されていることに注意する．

説明しても納得には時間がかかるかもしれない．図を丁寧に追いかけて理解することを望む．ほぼ同等の結果を次のプログラムで表示できるようにしてあるので，グラフを変えてみたりしながら理解して頂きたい．  
これが理解できた瞬間に，どうして動的計画法を普段なかなか人間は思いついて実行したりしないかを実感できることであろう．

**実装**

プログラミングに当たっては難しいところはない．  


![da2020-floyd-bitmap](https://user-images.githubusercontent.com/45651568/84224666-1817d000-ab18-11ea-8b78-ab412dda3675.jpg)


# Flyodのアルゴリズムに基づくプログラム

**目標**

正値重みの辺で構成される有向グラフにおいて，グラフ上の任意の頂点対の最小経路値とその経路を求めるＣプログラムを作成する．

**説明**

Floydのアルゴリズム本体は一つのユーザ定義関数 find_shortest_floyd() で実装する．

**コード**

C言語では無限大という数は扱えない．それに関連する演算「無限大 + 定数 = 無限大」も扱えない．
ここでは，無限大の代わりに十分大きな数（より正確には処理の途中で一時的に得られるであろう暫定経路値の最大値より大きい数）NCを用意し，NCに辺の最大重みを加えてもまだC言語の演算範囲内にあるものとして実装を行う．

なお，なにかしら引数を１つ指定すると，上述の解説図と同じようなシンプルな出力が得られるようにしてある．(simpledisplay変数に注意してコードを読むこと）


**備考**

最短経路を実際に求めている部分は本当に解いているのか？と疑いたくなるぐらい短いプログラムである．理解のためにコメント行と表示用コードを差し込んでいるが，試しにそれらを削除してみるとその短さが実感できるであろう．３重ループの内側にif文が1つあるだけなのである．それにも関わらずDijkstraのアルゴリズムよりも多くのことを実現できている．  
これは，如何にFloydのアルゴリズムがコンピュータとC言語に相性がよいかという証と言えよう．  

Dijkstraのときのような集合を使わないので，集合用のライブラリの構築と使用も必要ない．

In [ ]:
%%writefile shortest-floyd_J.c
// Shortest path by Floyd
//    2020/06/07 kameda
#include <stdio.h> // printf()
#include <stdlib.h> // atoi()
#include "graph6.h" // edge[][], N, NC

int simpledisplay = 0; // 0:normal, 1:simpler

// 頂点のための構造体変数
typedef struct {
	int cost; // 暫定経路値（暫定最小値）
	int via; // 経由頂点
} CostVia;

CostVia pathinfo[N][N]; // 	[a][b] 開始頂点a, 到着頂点b

// 全経路の表示（再帰呼出を利用）
// この方法だと最終到着頂点は表示されないので注意
void outputpath(int SrcNode, int DstNode){
	// 再帰呼び出しの終了条件は「直行便」 (via[s][d]==s)。
    // 出発側のみ表示
	if (SrcNode == pathinfo[SrcNode][DstNode].via) {
		printf("%2d ", SrcNode);
	} else {
		// 経由地の前後のPATHそれぞれ探索して表示
		outputpath(SrcNode, pathinfo[SrcNode][DstNode].via);
		outputpath(pathinfo[SrcNode][DstNode].via, DstNode);
	}
}

// 経由行列の表示
void print_path_status(void){
	int SrcNode, DstNode;

	if (simpledisplay == 0) {
		for (SrcNode = 0; SrcNode < N; SrcNode++) {
			for (DstNode = 0; DstNode < N; DstNode++) {
				if (pathinfo[SrcNode][DstNode].cost >= NC) 
					printf("---(--) ");
				else
					printf("%3d(%2d) ", pathinfo[SrcNode][DstNode].cost, pathinfo[SrcNode][DstNode].via);
			}
			printf("\n");
		}
	} else {
		for (SrcNode = 0; SrcNode < N; SrcNode++) {
			for (DstNode = 0; DstNode < N; DstNode++) {
				if (pathinfo[SrcNode][DstNode].cost >= NC) 
					printf("NC\t");
				else
					printf("%d\t", pathinfo[SrcNode][DstNode].cost);
				}
			printf("\n");
		}
	}

	return ;
}

// Floydのアルゴリズムの本体 
void find_shortest_floyd(void){
	int SrcNode, DstNode;
	int MidNode; // a.k.a. "k"
	int NewCost;
	
	// 直行のみを認める形で初期化
	// 直行できない頂点対の場合はNC（＝直交で行けない）という重みで初期化
	for (SrcNode = 0; SrcNode < N; SrcNode++) {
		for (DstNode = 0; DstNode < N; DstNode++) {
			pathinfo[SrcNode][DstNode].cost = edge[SrcNode][DstNode];
			pathinfo[SrcNode][DstNode].via  = SrcNode; // 直行が解の時は via == SrcNode
		}
	}
	printf("直通のみを認めた結果 ===\n");
	print_path_status();

    // 最外ループは step 3. と 5. に相当
    // ２番目と３番目のループは step 4. に相当
	for (MidNode = 0; MidNode < N; MidNode++) {
		for (SrcNode = 0; SrcNode < N; SrcNode++) {
			for (DstNode = 0; DstNode < N; DstNode++) {
				NewCost = pathinfo[SrcNode][MidNode].cost + pathinfo[MidNode][DstNode].cost;
				if (NewCost < pathinfo[SrcNode][DstNode].cost) {
					pathinfo[SrcNode][DstNode].cost = NewCost;
					pathinfo[SrcNode][DstNode].via  = MidNode;
				}
			}
		}
		printf("Step 4. 終了 (頂点 %2d を経由頂点として追加した結果) ===\n", MidNode);
		print_path_status();
	}

	// 結果の表示
	if (simpledisplay == 0) {
		printf("最短経路問題の答\n");
		for (SrcNode = 0; SrcNode < N; SrcNode++) {
			for (DstNode = 0; DstNode < N; DstNode++) {
				printf("%2d => %2d: ",SrcNode, DstNode);
				if (pathinfo[SrcNode][DstNode].cost == NC) {
					printf("  - : (No path)");
				} else {
					printf("%3d :", pathinfo[SrcNode][DstNode].cost);
					outputpath(SrcNode, DstNode);
					printf("%2d", DstNode);
				}
				printf("\n");
			}

			printf("\n");
		}
	}

	return ;
}

// Main function
int main(int argc, char *argv[]){
	  
    if (argc >= 2) {
		simpledisplay = 1;
		printf("結果を簡約表示のみにします．\n");
	}

	find_shortest_floyd();

	return 0;
}



コンパイルしてエラーが無いことを確認．

In [ ]:
!gcc -Wall -o shortest-floyd_J shortest-floyd_J.c

実行．

In [ ]:
!echo "通常版（途中結果で経由頂点も表示）----------------------------"
!./shortest-floyd_J

コマンドライン引数をなにか付けると，途中の表示がシンプルになる．

In [ ]:
!echo "シンプル版（算定経路値のみ表示）--------------------"
!./shortest-floyd_J a

# 節末課題

1. 最短経路の表示の仕組み  
shortest_floyd Cプログラムにおいて，全ての頂点対の最小経路を表示する部分には再帰呼出関数を用いている．これについて，本プログラムが最小経路を表示している仕組みを説明せよ．特に，via[][]が格納している値について言及しながら説明すること．（参考：去年まではこの部分を自ら作成するのが課題であった．今年から読むだけにしたので，随分簡単な課題にしたものである）


2. 計算量  
shortest_floyd プログラムの時間計算量と空間計算量を議論せよ．

3. shortest_diskstra プログラムの再利用  
N対Nの最短経路問題を解くにあたって，shortest_diskstra プログラムを改良して実現して見よ．プログラム内で開始頂点を1からNまで変更しながら，dijkstraのアルゴリズムを実行するという方針である．改良したプログラムについて，時間計算量と空間計算量を議論せよ．

4. 十分大きな数NC  
あるグラフが与えられた時，暫定経路値の一時的最大値と最大重み辺が分かれば，NCを決めることができる．これを正確に求めることは難しいが，アルゴリズムの趣旨を考えれば，これは正確な値である必要はない．そこで，グラフの隣接行列が与えられたとして，NCの近似値として役に立つ数字を 時間計算量 O(N^2) で求める簡単な方法を考えてみよ．


# 出典

筑波大学工学システム学類  
データ構造とアルゴリズム  
担当：亀田能成  
2021/05/26 初版  
